In [1]:
# %%capture
!pip install transformers>=4.41.2 accelerate>=0.31.0

## Loading the LLM

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda


# The Inputs and Outputs of a Trained Transformer LLM


In [3]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in


In [4]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

# Choosing a single token from the probability distribution (sampling / decoding)

In [5]:
prompt = "The capital of France is"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Tokenize the input prompt
input_ids = input_ids.to("cuda")

# Get the output of the model before the lm_head
model_output = model.model(input_ids)

# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])

In [6]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

In [7]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [8]:
lm_head_output.shape

torch.Size([1, 5, 32064])

In [19]:
from transformers import AutoTokenizer
import torch

# Define a prompt
prompt = "The capital of France is"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Pass the input through the model
with torch.no_grad():
    # Get the model's outputs including the hidden states
    model_output = model(input_ids, output_hidden_states=True)
    hidden_states = model_output.hidden_states[-1]  # Last hidden state
    logits = model.lm_head(hidden_states)           # Output of the lm_head

# Logits shape: [batch_size, seq_len, vocab_size]
print(f"Logits shape: {logits.shape}")

# Get the logits for the last token
last_token_logits = logits[0, -1, :]

# Convert logits to probabilities
probabilities = torch.softmax(last_token_logits, dim=-1)

# Decode the top-k predicted tokens for the next word
top_k = 15
top_k_probs, top_k_indices = torch.topk(probabilities, top_k)

print("\nTop-k Predictions for Next Token:")
for i, (token_id, prob) in enumerate(zip(top_k_indices, top_k_probs)):
    token = tokenizer.decode(token_id.item())
    print(f"{i+1}. {token} ({prob.item():.4f})")

# Optional: Display raw logits for the last token
print("\nRaw logits for the last token:")
print(last_token_logits)


Logits shape: torch.Size([1, 5, 32064])

Top-k Predictions for Next Token:
1. Paris (0.8750)
2. _ (0.0264)
3. not (0.0125)
4. ... (0.0076)
5. __ (0.0059)
6. 
 (0.0059)
7. known (0.0059)
8. a (0.0046)
9. [ (0.0022)
10. : (0.0022)
11. located (0.0017)
12. ** (0.0017)
13. an (0.0013)
14. in (0.0013)
15. what (0.0013)

Raw logits for the last token:
tensor([27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000],
       device='cuda:0', dtype=torch.bfloat16)
